In [1]:
import ray

2024-03-02 22:08:49,231	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# ray.init("ray://127.0.0.1:10001")

ray.init("ray://a575a5762915a45979e7bbdc62d66d1f-780973796.ap-northeast-1.elb.amazonaws.com:10001")



SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.8.18
Ray version:,2.9.3
Dashboard:,http://192.168.1.24:8265


In [3]:
# Normal Ray code follows
@ray.remote
def do_work(x):
    from sklearn.datasets import load_breast_cancer
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    data = load_breast_cancer()
    X = data.data
    y = data.target
    return X

result = do_work.remote(2)


In [4]:
result

ClientObjectRef(67a2e8cfa5a06db3ffffffffffffffffffffffff0200000001000000)

In [5]:
result_value = ray.get(result)

In [6]:
result_value


array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import ray
from ray import tune
from ray.tune.sklearn import TuneSearchCV

# データセットのロード
data = load_breast_cancer()
X = data.data
y = data.target

# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Rayの初期化
# ray.init("ray://127.0.0.1:10001")


2024-03-02 22:09:00,118	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:

# パラメータグリッドの定義
param_grid = {"max_depth": [None, 10, 20, 30]}

# TuneSearchCVの作成
tune_search = TuneSearchCV(RandomForestClassifier(n_estimators=100),
                           param_distributions=param_grid,
                           n_trials=10,
                           early_stopping=True,
                           max_iters=10,
                           search_optimization="random"
                           )

# モデルの訓練
tune_search.fit(X_train, y_train)

# テストデータでの予測
predictions = tune_search.predict(X_test)

# 精度の計算
accuracy = accuracy_score(y_test, predictions)

print(f"Test Accuracy: {accuracy}")


(run pid=200) [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(run pid=200) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.
(run pid=200) /home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/execution/tune_controller.py:190: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
(run pid=200)   warnings.warn(


(run pid=200) ╭───────────────────────────────────────────────────────────────────╮
(run pid=200) │ Configuration for experiment     _Trainable_2024-03-02_05-09-04   │
(run pid=200) ├───────────────────────────────────────────────────────────────────┤
(run pid=200) │ Search algorithm                 BasicVariantGenerator            │
(run pid=200) │ Scheduler                        AsyncHyperBandScheduler          │
(run pid=200) │ Number of trials                 4                                │
(run pid=200) ╰───────────────────────────────────────────────────────────────────╯
(run pid=200) 
(run pid=200) View detailed results here: /home/ray/ray_results/_Trainable_2024-03-02_05-09-04
(run pid=200) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ray/ray_results/_Trainable_2024-03-02_05-09-04`
(run pid=200) 


Test Accuracy: 0.9649122807017544
